Appender file

In [ ]:
import pandas as pd 
import numpy as np 
import geopandas as gpd 
import xarray as xr 
import rioxarray as rxr

Master file - contains a template for Geojson \
df - reads the geojson file from planner, adds some columns, formats it \
df and master file is merged and updated file is written back to master file 

In [ ]:
# Reading master file
master = gpd.read_file('Master_file')
master
# Determine the starting ID based on the master file
if not master.empty and 'id' in master.columns:
    # Ensure we get a valid maximum id (if the id column is numeric)
    max_id = master['id'].max() if pd.notna(master['id'].max()) else 0
else:
    max_id = 0

# Read the new file here
df = gpd.read_file('Planner_output.geojson') # the file name has to be dynamic as per planner files
df['centroid']=df.to_crs('+proj=cea').centroid.to_crs(df.crs)
df['simulation_status'] = None
df['completion_date'] = pd.NaT  # Setting it as a datetime type
df['expiration_date'] = pd.NaT
df['satellite'] = None
df.insert(0, 'id', range(max_id + 1, max_id + 1 + len(df)))
# df.insert(0, 'id', range(1, len(df) + 1))
df['polygon_groundtrack'] = None
df['latitude'] = df['centroid'].y
df['longitude'] = df['centroid'].x
df['Planner_geometry'] = df['geometry'].to_wkt()
df['centroid'] = df['centroid'].to_wkt()
df.drop('geometry', axis=1, inplace=True)
df.rename(columns={'polygon_groundtrack': 'geometry'}, inplace=True)
# Order
df = df.loc[:, ['id','time' ,'final_eta', 'Planner_geometry','centroid','latitude','longitude','expiration_date','simulation_status','completion_date','satellite','geometry']]
combined_gdf = pd.concat([master, df], ignore_index=True)
combined_gdf = combined_gdf.set_crs(epsg=4326)
combined_gdf.to_file("Master_file", driver="GeoJSON")
combined_gdf